# Dhyuti Walkthroughs - PINNs for inverse problems through 1D analogs of various domains
Author: Rahul Sundar (2025), Dhyuti

Purpose: Generate analytic/synthetic data for PINN pretraining


# 🔥 Physics-Informed Neural Networks (PINNs) — A Hands-on Intuition Builder

Welcome!  
This notebook is designed for a **multi-domain audience** — whether you come from  
- 🌊 Fluid Dynamics  
- 🧱 Material Science  
- 🧮 Nonlinear Dynamics  
- 💰 Quantitative Finance  
- 🧫 Computational Biology  
- 🌐 Geospatial & Climate Science  
- ⚡ Electromagnetics & Field Inversions  
- 🚗 EV & Engineering Design  

Here, you’ll find a **simple 1D differential equation analog** that you can visualize and use to understand how PINNs bridge physics and data.

Each domain’s example corresponds to a simple **closed-form (analytic)** 1D PDE/ODE that we can visualize and later feed into a PINN training workflow.

Let’s begin!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from ipywidgets import interact, Dropdown, FloatSlider, fixed
import math

# ⚙️ Domain Equations and Analytical Solutions

Below are **1D analogs** representing the *essence* of various complex systems across disciplines.  
Each equation expresses the governing physical, biological, or financial law that can be encoded in a **Physics-Informed Neural Network (PINN)**.

---

| **Domain** | **Representative Equation** | **Description** |
|-------------|-----------------------------|-----------------|
| **Material Science** | $$\frac{\partial u}{\partial t} = D \frac{\partial^2 u}{\partial x^2}$$ | Models heat or atom diffusion through a medium. |
| **Fluid Dynamics** | $$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial x^2}$$ | 1D analog of the Navier–Stokes equation, showing nonlinear advection and viscous effects. |
| **Nonlinear Dynamics** | $$\frac{d^2 u}{d t^2} + \delta \frac{d u}{d t} + \alpha u + \beta u^3 = 0$$ | The Duffing oscillator: nonlinear restoring force model. |
| **Investment Banking** | $$\frac{\partial V}{\partial t} + \frac{1}{2}\sigma^2 S^2 \frac{\partial^2 V}{\partial S^2} + rS \frac{\partial V}{\partial S} - rV = 0$$ | The Black–Scholes PDE for option pricing. |
| **Computational Biology** | $$\frac{d u}{d t} = r u \left(1 - \frac{u}{K}\right)$$ | Logistic growth model showing population saturation. |
| **Graph Theory (Diffusion)** | $$\frac{d u}{d t} = L u$$ | Diffusion on a graph, simplified here as a 1D Laplacian. |
| **Geospatial Sciences** | $$\frac{\partial u}{\partial t} + c \frac{\partial u}{\partial x} = D \frac{\partial^2 u}{\partial x^2}$$ | Advection–diffusion of heat or pollutants. |
| **Electromagnetics** | $$\frac{\partial^2 E}{\partial x^2} = \frac{1}{c^2}\frac{\partial^2 E}{\partial t^2}$$ | 1D wave equation describing field propagation. |
| **EV Domain** | $$\frac{dV}{dt} + \frac{V}{RC} = \frac{V_{\text{in}}(t)}{RC}$$ | RC circuit charge/discharge model for EV systems. |
| **Climate Modeling** | $$C \frac{dT}{dt} = S(1 - \alpha) - \varepsilon \sigma T^4$$ | Global mean energy balance for Earth’s temperature. |
| **Engineering Design** | $$\frac{d^2 y}{d x^2} = \frac{M(x)}{E I}$$ | Beam bending equation representing elastic deformation. |

---

💡 **Note:**  
Each equation can serve as the governing constraint in a **PINN**, where instead of solving directly via discretization, the neural network learns a function $ u(x, t) $ (or $ V(t)$, $ T(t) $, etc.) that *minimizes both data loss and physics loss*.


**Material Science Setup:**  
- **1D Analog:** Heat diffusion along a rod.  
- **Domain:** $ x \in [0, 1] $, $ t \in [0, 1] $  
- **Initial Condition:** $ u(x, 0) = \sin(\pi x) $  
- **Boundary Conditions:** $ u(0, t) = u(1, t) = 0 $  
- **Analytical Solution:** $ u(x, t) = e^{-\pi^2 D t} \sin(\pi x) $  
- **Applications:** Thermal conduction in solids, atomic diffusion, sintering, semiconductor doping, corrosion modeling.

**Fuid Dynamics setup:**  
- **1D Analog:** Flow velocity in a viscous channel.  
- **Domain:** $ x \in [-1, 1] $, $ t \in [0, 1] $
- **Initial Condition:** $ u(x, 0) = -\sin(\pi x) $  
- **Boundary Conditions:** $ u(-1, t) = u(1, t) = 0 $  
- **Analytical Solution:** Approximate with Cole–Hopf transformation.  
- **Applications:** Shock formation, turbulence modeling, aerodynamic drag estimation, fluid–structure interactions.


**Nonlinear dynamics setup:**  
- **1D Analog:** Mass on a nonlinear spring.  
- **Domain:** $ t \in [0, 20] $
- **Initial Condition:** $ u(0) = 1, \ \dot{u}(0) = 0 $  
- **Boundary Conditions:** None (time evolution problem).  
- **Applications:** Nonlinear vibrations in beams, MEMS, climate oscillations, neuronal models, forced oscillators in electronics.


**Financial systems / Investment banking example setup:**  
- **1D Analog:** Option price as function of asset price $ S $ and time $ t $.  
- **Domain:** $ S \in [0, 1], \ t \in [0, 1] $  
- **Initial Condition:** $ V(S, 0) = \max(S - K, 0) $ (call option payoff).  
- **Boundary Conditions:** $ V(0, t) = 0, \ V(1, t) = 1 - e^{-r t} $.  
- **Analytical Solution:** Black–Scholes formula.  
- **Applications:** Derivative pricing, risk modeling, financial forecasting, hedging strategy optimization.


**Computational Biology example setup:**  
- **1D Analog:** Population density vs. time.  
- **Domain:** $ t \in [0, 10] $  
- **Initial Condition:** $ u(0) = 0.1K $
- **Analytical Solution:** $ u(t) = \frac{K}{1 + A e^{-r t}} $, where $ A = \frac{K - u_0}{u_0} $.  
- **Applications:** Bacterial population, tumor growth, ecological dynamics, epidemic spread.


**Graph theory setup:**  
- **1D Analog:** Linear diffusion chain.  
- **Domain:** $ x \in [0, 1], \ t \in [0, 1] $  
- **Initial Condition:** $ u(x, 0) = \sin(\pi x) $
- **Boundary Conditions:** Dirichlet zero boundaries.  
- **Applications:** Network heat diffusion, consensus dynamics, random walks on graphs, community detection.


**Geospatial sciences setup:**  
- **1D Analog:** Pollutant transport in a river.  
- **Domain:** $ x \in [0, 1], \ t \in [0, 1] $
- **Initial Condition:** $ u(x, 0) = e^{-(x - 0.5)^2 / 0.01} $  
- **Boundary Conditions:** $ u(0, t) = u(1, t) = 0 $  
- **Analytical Solution:** Gaussian pulse with drift.  
- **Applications:** Climate tracer transport, groundwater flow, pollutant modeling, erosion prediction.

**Electromagnetics setup:**  
- **1D Analog:** Vibrating string or EM wave along a wire.  
- **Domain:** $ x \in [0, 1], \ t \in [0, 2] $  
- **Initial Condition:** $ E(x, 0) = \sin(\pi x), \ \partial_t E(x, 0) = 0 $  
- **Boundary Conditions:** $ E(0, t) = E(1, t) = 0 $.  
- **Analytical Solution:** $ E(x, t) = \sin(\pi x) \cos(\pi c t) $.  
- **Applications:** Signal propagation, microwave resonances, antenna field simulation.


**Electric vehicles use case setup:**  
- **1D Analog:** Charging capacitor through resistor.  
- **Domain:** $ t \in [0, 10] $  
- **Initial Condition:** $ V(0) = 0 $  
- **Analytical Solution:** $ V(t) = V_{\text{in}}(1 - e^{-t/(RC)}) $.  
- **Applications:** EV battery models, thermal analogs, BMS simulations.


**Climate modelling setup:**  
- **1D Analog:** Earth’s mean temperature over time.  
- **Domain:** $ t \in [0, 100] $  
- **Initial Condition:** $ T(0) = 288\text{ K} $  
- **Analytical Solution:** Approximate via steady-state $ T = \left(\frac{S(1 - \alpha)}{\varepsilon \sigma}\right)^{1/4} $.  
- **Applications:** Climate prediction, sensitivity to albedo or CO₂ forcing, planetary habitability.


**Engineering design setup:**  
- **1D Analog:** Uniform beam under central load.  
- **Domain:** $ x \in [0, 1] $
- **Boundary Conditions:** $ y(0) = y(1) = 0 $  
- **Analytical Solution:** $ y(x) = \frac{M_0 x (1 - x)}{2 E I} $.  
- **Applications:** Structural optimization, aerospace and mechanical design, deformation analysis.

---

🌍 **Summary Insight:**  
Each of these simple **1D analogs** hides a vast world of physics, finance, and biology beneath them.  
By encoding these equations into **PINN loss functions**, we can recover missing data, infer parameters, or perform forward simulations *without explicit meshing*, making them a perfect entry point for your audience across domains.



In [ ]:
# --- 1D Heat Equation (u_t = α u_xx) ---
def heat_equation(x, t, alpha=0.01, L=1.0):
    return np.exp(-alpha * (np.pi**2 / L**2) * t) * np.sin(np.pi * x / L)

# --- Burgers’ Equation (approx analytic form) ---
def burgers_equation(x, t, nu=0.01/np.pi):
    phi = np.exp(-(x - 4*t)**2 / (4*nu*(t+1))) + np.exp(-(x - 4*t - 2*np.pi)**2 / (4*nu*(t+1)))
    u = -2*nu*(phi**(-1))*((-(x - 4*t)/(2*nu*(t+1)))*np.exp(-(x - 4*t)**2/(4*nu*(t+1))) +
                           (-(x - 4*t - 2*np.pi)/(2*nu*(t+1)))*np.exp(-(x - 4*t - 2*np.pi)**2/(4*nu*(t+1))))
    return u

# --- Advection–Diffusion Gaussian Packet ---
def advection_diffusion(x, t, c=1.0, D=0.01):
    return np.exp(-((x - c*t)**2) / (4*D*(t + 1))) / np.sqrt(4*np.pi*D*(t + 1))

# --- Duffing Oscillator (simple approximation) ---
def duffing_oscillator(t, delta=0.2, alpha=-1, beta=1, gamma=0.3, omega=1.2):
    return np.exp(-delta*t)*(np.cos(omega*t) + 0.5*np.sin(omega*t))

# --- Ornstein–Uhlenbeck process (mean function only) ---
def ornstein_uhlenbeck(t, mu=0.0, theta=0.7, x0=1.0):
    return mu + (x0 - mu)*np.exp(-theta*t)

# --- Eigenvalue / time-growth problem example ---
def exponential_growth(t, lam=0.5):
    return np.exp(lam*t)

# --- Steady-state Poisson / Beam deflection ---
def poisson_steady(x, L=1.0):
    return 0.5*(x*(L - x))

# --- Biological / diffusion steady-state (logistic shape) ---
def bio_steady(x, k=10):
    return 1 / (1 + np.exp(-k*(x - 0.5)))

## 🧮 Step 3 — Create a 1D Space-Time Grid

We’ll define a uniform grid for `(x, t)` that can be reused across all PDEs.

## 🎨 Step 4 — Plot Solutions

We’ll use a color mesh to visualize how the field evolves in space and time for each PDE.


In [ ]:
def generate_grid_1D(nx=100, nt=100, tmax=1.0):
    x = np.linspace(0, 1, nx)
    t = np.linspace(0, tmax, nt)
    X, T = np.meshgrid(x, t)
    return X, T

def plot_solution(X, T, U, title=""):
    fig = plt.figure(figsize=(6,4))
    cp = plt.pcolormesh(X, T, U, shading='auto', cmap=cm.viridis)
    plt.colorbar(cp)
    plt.xlabel("x")
    plt.ylabel("t")
    plt.title(title)
    plt.show()


def visualize_equation(eq_name, nx=100, nt=100, noise=0.0):
    X, T = generate_grid_1D(nx, nt)

    if eq_name == "Heat Equation":
        U = heat_equation(X, T)
    elif eq_name == "Burgers Equation":
        U = burgers_equation(X, T)
    elif eq_name == "Advection–Diffusion":
        U = advection_diffusion(X, T)
    elif eq_name == "Duffing Oscillator":
        X, T = np.meshgrid(np.linspace(0, 1, nx), np.linspace(0, 10, nt))
        U = duffing_oscillator(T)
    elif eq_name == "Ornstein–Uhlenbeck":
        X, T = np.meshgrid(np.linspace(0, 1, nx), np.linspace(0, 10, nt))
        U = ornstein_uhlenbeck(T)
    elif eq_name == "Exponential Growth":
        X, T = np.meshgrid(np.linspace(0, 1, nx), np.linspace(0, 5, nt))
        U = exponential_growth(T)
    elif eq_name == "Poisson Steady State":
        X, T = np.meshgrid(np.linspace(0, 1, nx), np.zeros(nt))
        U = poisson_steady(X)
    elif eq_name == "Bio Steady State":
        X, T = np.meshgrid(np.linspace(0, 1, nx), np.zeros(nt))
        U = bio_steady(X)
    else:
        raise ValueError("Unknown equation name")

    if noise > 0:
        U += noise * np.random.randn(*U.shape)

    plot_solution(X, T, U, title=eq_name)


## 🧠 Step 5 — Interactive PINN Domain Explorer

Use the dropdown to pick a domain.  
Each domain corresponds to a physical process with a **different governing equation**.


In [ ]:
interact(
    visualize_equation,
    eq_name=Dropdown(
        options=["Heat Equation", "Burgers Equation", "Advection–Diffusion",
                 "Duffing Oscillator", "Ornstein–Uhlenbeck",
                 "Exponential Growth", "Poisson Steady State", "Bio Steady State"],
        value="Heat Equation",
        description="Equation:"
    ),
    nx=fixed(150),
    nt=fixed(150),
    noise=FloatSlider(min=0, max=0.2, step=0.01, value=0.0, description="Noise")
);


## 💾 Step 6 — Save Synthetic Data for PINN Training

You can export the grid and corresponding `U(x, t)` field as `.npz` files.
These can then be loaded into PyTorch/TensorFlow for PINN training.


In [ ]:
def export_solution(eq_name, nx=100, nt=100, filename="synthetic_data.npz"):
    X, T = generate_grid_1D(nx, nt)
    eq_funcs = {
        "Heat Equation": heat_equation,
        "Burgers Equation": burgers_equation,
        "Advection–Diffusion": advection_diffusion,
        "Duffing Oscillator": lambda x, t: duffing_oscillator(t),
        "Ornstein–Uhlenbeck": lambda x, t: ornstein_uhlenbeck(t),
        "Exponential Growth": lambda x, t: exponential_growth(t),
        "Poisson Steady State": lambda x, t: poisson_steady(x),
        "Bio Steady State": lambda x, t: bio_steady(x),
    }
    U = eq_funcs[eq_name](X, T)
    np.savez(filename, X=X, T=T, U=U)
    print(f"✅ Saved {filename}")

def load_solution(filename="synthetic_data.npz"):
    data = np.load(filename)
    print("🔍 Keys:", data.files)
    return data["X"], data["T"], data["U"]

# Example usage:
# export_solution("Heat Equation", nx=100, nt=100)
# X, T, U = load_solution()


## 🔗 Step 7 — Preparing Data for a PINN

If you’d like to feed this into a Physics-Informed Neural Network,  
you can convert it into tensors suitable for PyTorch.


In [ ]:
import torch

def prepare_torch_data(eq_name, nx=100, nt=100):
    X, T = generate_grid_1D(nx, nt)
    eq_funcs = {
        "Heat Equation": heat_equation,
        "Burgers Equation": burgers_equation,
        "Advection–Diffusion": advection_diffusion,
        "Duffing Oscillator": lambda x, t: duffing_oscillator(t),
        "Ornstein–Uhlenbeck": lambda x, t: ornstein_uhlenbeck(t),
        "Exponential Growth": lambda x, t: exponential_growth(t),
        "Poisson Steady State": lambda x, t: poisson_steady(x),
        "Bio Steady State": lambda x, t: bio_steady(x),
    }
    U = eq_funcs[eq_name](X, T)
    x_tensor = torch.tensor(X.flatten(), dtype=torch.float32).reshape(-1, 1)
    t_tensor = torch.tensor(T.flatten(), dtype=torch.float32).reshape(-1, 1)
    u_tensor = torch.tensor(U.flatten(), dtype=torch.float32).reshape(-1, 1)
    print(f"✅ Data ready for {eq_name}: shape =", u_tensor.shape)
    return x_tensor, t_tensor, u_tensor

# Example:
# x, t, u = prepare_torch_data("Heat Equation")


# 🧠 Step 8 — Physics-Informed Neural Network (PINN) Example for the Heat Equation

Let’s demonstrate a minimal PINN example for the **1D Heat Equation**:

$$ \frac{\partial u}{\partial t} = \alpha \frac{\partial^2 u}{\partial x^2}, \quad x \in [0, 1], \ t \in [0, 1] $$

**Boundary/Initial Conditions**
$$ u(0, t) = 0, \quad u(1, t) = 0, \quad u(x, 0) = \sin(\pi x) $$

**Analytic Solution**
$$ u(x, t) = e^{-\alpha \pi^2 t} \sin(\pi x) $$

Domain: $x \in [0,1], \ t \in [0,1]$

Boundary Conditions: $u(0,t)=u(1,t)=0$

Initial Condition: $u(x,0)=\sin(\pi x)$

True Parameters: $D=0.01, \ k=0.5$

Inverse Objective: Infer $(D,k)$ from sparse, noisy observations of $u(x,t)$

We’ll train a small neural network to **learn and reconstruct $u(x,t)$ and the unknown parameters $(D, k)$**,  
while ensuring that the **governing PDE** is satisfied at collocation points.



In [ ]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

# true params
D_true, k_true = 0.01, 0.5

# domain
nx, nt = 100, 100
x = np.linspace(0, 1, nx)
t = np.linspace(0, 1, nt)
X, T = np.meshgrid(x, t)

# analytic solution (separation of variables)
u_true = np.exp(-(np.pi**2*D_true + k_true)*T) * np.sin(np.pi*X)

# add Gaussian noise
noise_level = 0.02
u_noisy = u_true + noise_level * np.random.randn(*u_true.shape)

# apply random masking (simulate missing sensors)
mask = np.random.rand(*u_true.shape) < 0.2   # 20% observed
u_obs = np.where(mask, u_noisy, np.nan)

# convert to tensors
x_torch = torch.tensor(X.flatten()[:,None], dtype=torch.float32)
t_torch = torch.tensor(T.flatten()[:,None], dtype=torch.float32)
u_torch = torch.tensor(u_noisy.flatten()[:,None], dtype=torch.float32)

# plot
plt.figure(figsize=(5,4))
plt.imshow(u_obs, extent=[0,1,0,1], origin='lower', aspect='auto')
plt.title("Noisy & Masked Observations of u(x,t)")
plt.xlabel("x"); plt.ylabel("t")
plt.colorbar(label="u")
plt.show()


In [ ]:
# simple MLP for u(x,t)
class PINN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 64), nn.Tanh(),
            nn.Linear(64, 64), nn.Tanh(),
            nn.Linear(64, 64), nn.Tanh(),
            nn.Linear(64, 1)
        )
        # learnable physical parameters
        self.D = nn.Parameter(torch.tensor(0.005))  # initial guess
        self.k = nn.Parameter(torch.tensor(0.1))

    def forward(self, x, t):
        X = torch.cat([x, t], dim=1)
        return self.net(X)

# instantiate model
model = PINN()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# collocation points (physics)
n_col = 5000
x_col = torch.rand(n_col, 1)
t_col = torch.rand(n_col, 1)


# observed points (subset)
idx_obs = np.where(~np.isnan(u_obs.flatten()))[0]
x_obs = x_torch[idx_obs]
t_obs = t_torch[idx_obs]
u_obs_t = u_torch[idx_obs]

## ⚙️ Step 9 — Define the Loss Function

The total loss is a combination of:
1. **Data Loss** — difference between predicted and known boundary/initial values.  
2. **Physics Loss** — PDE residual ensuring the model obeys the heat equation:
   $$  f = u_t - \alpha u_{xx} = 0 $$


In [ ]:
# define physics residual
def pde_residual(model, x, t):
    x.requires_grad_(True)
    t.requires_grad_(True)
    u = model(x, t)
    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    return u_t - model.D * u_xx + model.k * u  # residual form

## 🚀 Step 10 — Train the PINN

Let’s train for ~2000 iterations and visualize convergence.


In [ ]:
lambda_d, lambda_p = 1.0, 0.1

for epoch in range(4000):
    optimizer.zero_grad()

    # data loss
    u_pred_obs = model(x_obs, t_obs)
    loss_data = torch.mean((u_pred_obs - u_obs_t)**2)

    # physics loss
    res = pde_residual(model, x_col, t_col)
    loss_phys = torch.mean(res**2)

    loss = lambda_d * loss_data + lambda_p * loss_phys
    loss.backward()
    optimizer.step()

    if epoch % 500 == 0:
        print(f"Epoch {epoch} | Total: {loss.item():.6f} | D: {model.D.item():.4f} | k: {model.k.item():.4f}")

## 🌈 Step 11 — Visualize Learned Solution

Let’s compare the PINN’s output to the analytic solution.

In [ ]:
# # predicted field
# with torch.no_grad():
#     u_pred = model(x_torch, t_torch).cpu().numpy().reshape(nt, nx)

# plt.figure(figsize=(6,4))
# plt.imshow(u_pred, extent=[0,1,0,1], origin='lower', aspect='auto')
# plt.title("PINN Reconstruction of u(x,t)")
# plt.xlabel("x"); plt.ylabel("t")
# plt.colorbar(label="u_pred")
# plt.show()

# print(f"Learned D = {model.D.item():.4f}, True D = {D_true}")
# print(f"Learned k = {model.k.item():.4f}, True k = {k_true}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Evaluation grid
x_eval = torch.linspace(0, 1, 100).reshape(-1, 1).to(device)
t_eval = torch.linspace(0, 1, 100).reshape(-1, 1).to(device)
X_eval, T_eval = torch.meshgrid(x_eval.squeeze(), t_eval.squeeze(), indexing='ij')

x_eval_flat, t_eval_flat = X_eval.reshape(-1, 1), T_eval.reshape(-1, 1)
u_pred = model(x_eval_flat, t_eval_flat).detach().cpu().numpy().reshape(100, 100)
u_true = np.exp(-(np.pi**2*D_true + k_true)*T_eval.detach().cpu().numpy()) * np.sin(np.pi*X_eval.detach().cpu().numpy())


fig, axs = plt.subplots(1,3, figsize = (10,4))
im1 = axs[0].pcolormesh(X_eval.cpu(), T_eval.cpu(), u_true, cmap="hot", shading='auto')
axs[0].title.set_text("Analytic Solution")
plt.colorbar(im1, ax = axs[0])

im2 = axs[1].pcolormesh(X_eval.cpu(), T_eval.cpu(), u_pred, cmap="hot", shading='auto')
axs[1].title.set_text("PINN Prediction")
plt.colorbar(im2, ax = axs[1])


im3 = axs[2].pcolormesh(X_eval.cpu(), T_eval.cpu(), 100*np.abs(u_pred - u_true)/np.mean(np.abs(u_true)), cmap="BuPu", shading='auto')
axs[2].title.set_text("Percentage Relative Error")
plt.colorbar(im3, ax = axs[2])
plt.tight_layout()
plt.show()




# 🧭 Step 12 — Discussion and Extensions

**You’ve just trained your first PINN!**

Here’s what happened:
1. The network learned `u(x,t)` not just from data — but also from **physics constraints**.  
2. This enforces **generalization** even where no data is given (e.g., interior points).  
3. The method easily extends to:
   - **Material Science:** Diffusion or fracture PDEs  
   - **Fluid Dynamics:** Navier–Stokes surrogates  
   - **Finance:** Black–Scholes PDEs  
   - **Climate Models:** Advection–diffusion or radiative transfer  
   - **Electromagnetics:** Wave / Helmholtz equations  

Next, try swapping in your favorite domain PDE from the earlier dropdowns  
and see how you can formulate its PINN residual! ⚡

Each of the following can reuse the previous cells and you only change:
| Domain                 | Replace PDE Residual          | Learnable Params     | Notes                                |
| ---------------------- | ----------------------------- | -------------------- | ------------------------------------ |
| **Burgers’**           | $u_t + uu_x - νu_{xx}$        | $ν$ = nn.Parameter() | Increase collocation (~10k)          |
| **Duffing**            | $y_{tt} + cy_t + ky + βy^3$ | $c, β$               | Time-only domain                     |
| **Ornstein–Uhlenbeck** | $m_t - κ(θ - m)$             | $κ, θ$               | Fast 1D ODE                          |
| **Biology (source)**   | $u_t - Du_{xx} - S(x)$         | $S(x)$ via small NN  | Add smoothness regularizer           |
| **EM Poisson**         | $(εu_x)_x - s(x)$            | $s(x)$ via small NN  | Steady (no t term)                   |
| **Beam Bending**       | $w_{xxxx} - q(x)/(EI)$          | $q(x)$ via small NN  | Use `torch.autograd.grad` four times |
